In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
from pathlib import Path

pd.set_option("display.max_columns", 500)

## Creating Top10 HCPCS CMS PUF file

In [2]:
CY2017 = pd.read_csv("./data/Medicare_Provider_Util_Payment_PUF_CY2017.txt", sep="\t",header=0,skiprows=[1])

/Users/rad/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df = CY2017.copy()

In [4]:
df.columns

Index(['npi', 'nppes_provider_last_org_name', 'nppes_provider_first_name',
       'nppes_provider_mi', 'nppes_credentials', 'nppes_provider_gender',
       'nppes_entity_code', 'nppes_provider_street1', 'nppes_provider_street2',
       'nppes_provider_city', 'nppes_provider_zip', 'nppes_provider_state',
       'nppes_provider_country', 'provider_type',
       'medicare_participation_indicator', 'place_of_service', 'hcpcs_code',
       'hcpcs_description', 'hcpcs_drug_indicator', 'line_srvc_cnt',
       'bene_unique_cnt', 'bene_day_srvc_cnt', 'average_Medicare_allowed_amt',
       'average_submitted_chrg_amt', 'average_Medicare_payment_amt',
       'average_Medicare_standard_amt'],
      dtype='object')

In [5]:
df['Total_submitted_chrg_amt'] = df['line_srvc_cnt'] * df['average_submitted_chrg_amt']
df['Total_Medicare_payment_amt'] = df['line_srvc_cnt'] * df['average_Medicare_payment_amt']

In [6]:
df.head()

,npi,nppes_provider_last_org_name,nppes_provider_first_name,nppes_provider_mi,nppes_credentials,nppes_provider_gender,nppes_entity_code,nppes_provider_street1,nppes_provider_street2,nppes_provider_city,nppes_provider_zip,nppes_provider_state,nppes_provider_country,provider_type,medicare_participation_indicator,place_of_service,hcpcs_code,hcpcs_description,hcpcs_drug_indicator,line_srvc_cnt,bene_unique_cnt,bene_day_srvc_cnt,average_Medicare_allowed_amt,average_submitted_chrg_amt,average_Medicare_payment_amt,average_Medicare_standard_amt,Total_submitted_chrg_amt,Total_Medicare_payment_amt
0,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,215021854,MD,US,Internal Medicine,Y,F,99217,Hospital observation care discharge,N,100.0,96,100,73.398800,325.780000,56.827200,57.492400,32578.0,5682.72
1,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,215021854,MD,US,Internal Medicine,Y,F,99218,Hospital observation care typically 30 minutes,N,26.0,25,26,100.080000,449.000000,78.460000,79.306154,11674.0,2039.96
2,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,215021854,MD,US,Internal Medicine,Y,F,99219,Hospital observation care typically 50 minutes,N,52.0,51,52,136.380000,614.000000,102.807692,103.895385,31928.0,5346.00
3,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,215021854,MD,US,Internal Medicine,Y,F,99220,Hospital observation care typically 70 minutes...,N,59.0,59,59,190.363729,755.932203,141.293559,142.865763,44600.0,8336.32
4,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,215021854,MD,US,Internal Medicine,Y,F,99221,"Initial hospital inpatient care, typically 30 ...",N,16.0,16,16,101.680000,462.812500,79.710000,80.750000,7405.0,1275.36


In [7]:
Total_submitted = df.Total_submitted_chrg_amt.sum()

In [8]:
#per-row (per HCPS not grouped) --(wrong code)
top = df.sort_values(by='Total_submitted_chrg_amt', ascending=False).hcpcs_code.unique() 
x = df[df.hcpcs_code.isin(top[0:10])]
x.shape

(29976, 28)

In [9]:
x.hcpcs_code.value_counts()

88305    18547
A0427     7188
G0482     1277
G0483     1225
93229      693
R0070      342
A0436      303
A0431      303
G0249       94
81528        4
Name: hcpcs_code, dtype: int64

In [10]:
# sort by top 10 hcpcs:
top_hcpcs = df.groupby(['hcpcs_code'])['Total_submitted_chrg_amt'].sum().sort_values(ascending=False)
top10_hcpcs = top_hcpcs[0:10].index
top10_hcpcs

Index(['99214', '99213', '99285', '66984', '99232', '99233', 'A0427', '99223',
       '99291', '99284'],
      dtype='object', name='hcpcs_code')

In [11]:
df_top10 = df[df.hcpcs_code.isin(top10_hcpcs)]
df_top10.shape

(1520985, 28)

In [ ]:
df_top10.

In [12]:
df_top10.hcpcs_code.value_counts() #number of submitted charges
# bene_unique_cnt / total_bene_unique_cnt
# unique_npi / total_unique_npi

99213    457843
99214    428268
99232    175336
99223    133368
99233    120664
99291     62680
99284     60805
99285     57220
66984     17613
A0427      7188
Name: hcpcs_code, dtype: int64

In [17]:
pd.DataFrame({'Total_submitted_sum': top_hcpcs[1:10],\
              'Total_submitted_percent': ((top_hcpcs[0:10] / Total_submitted) * 100)\
              ''})

SyntaxError: invalid syntax (<ipython-input-17-69359378ef22>, line 1)

In [15]:
top_hcpcs[1:10]

hcpcs_code
99213    1.287074e+10
99285    1.244005e+10
66984    8.510616e+09
99232    7.573940e+09
99233    5.667375e+09
A0427    5.432417e+09
99223    4.681324e+09
99291    4.499186e+09
99284    3.748309e+09
Name: Total_submitted_chrg_amt, dtype: float64

In [16]:
top_hcpcs[0:10] / Total_submitted * 100

hcpcs_code
99214    6.438335
99213    4.076711
99285    3.940294
66984    2.695674
99232    2.398989
99233    1.795099
A0427    1.720677
99223    1.482774
99291    1.425083
99284    1.187249
Name: Total_submitted_chrg_amt, dtype: float64

In [18]:
df_top10.groupby('hcpcs_code')['npi'].count()

hcpcs_code
66984     17613
99213    457843
99214    428268
99223    133368
99232    175336
99233    120664
99284     60805
99285     57220
99291     62680
A0427      7188
Name: npi, dtype: int64

In [ ]:
#df_top10.to_csv(Path("data","top10_hcpcs.csv")) #create a new csv file

In [ ]:
#sort via highest avg Total_average_submitted_charge_amount
#pick top 10 
#find proportion for the top 10
# drop all columns not in top 10

# LEIE

In [19]:
#LEIE
LEIE = pd.read_csv('data/LEIE_08_2019.csv')

/Users/rad/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [20]:
#Top10-HCPCS
df = pd.read_csv('data/top10_hcpcs.csv', index_col=0)

/Users/rad/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/rad/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [22]:
LEIE.shape

(72476, 18)

In [23]:
LEIE['exyear'] = pd.to_datetime(LEIE["EXCLDATE"], format="%Y%m%d").dt.year
LEIE['exmonth'] = pd.to_datetime(LEIE["EXCLDATE"], format="%Y%m%d").dt.month

In [109]:
LEIE.columns = LEIE.columns.str.lower()
LEIE.head()

,lastname,firstname,midname,busname,general,specialty,upin,npi,dob,address,city,state,zip,excltype,excldate,reindate,waiverdate,wvrstate,exyear,exmonth
0,NaN,NaN,,14 LAWRENCE AVE PHARMACY,PHARMACY,NaN,NaN,0,NaN,14 LAWRENCE AVENUE,SMITHTOWN,NY,11787,1128a1,19880830,0,0,NaN,1988,8
1,NaN,NaN,,143 MEDICAL EQUIPMENT CO,DME COMPANY,DME - OXYGEN,NaN,0,NaN,701 NW 36 AVENUE,MIAMI,FL,33125,1128b7,19970620,0,0,NaN,1997,6
2,NaN,NaN,,184TH STREET PHARMACY CORP,OTHER BUSINESS,PHARMACY,NaN,1922348218,NaN,69 E 184TH ST,BRONX,NY,10468,1128a1,20180419,0,0,NaN,2018,4
3,NaN,NaN,,1951 FLATBUSH AVENUE PHARMACY,PHARMACY,NaN,NaN,0,NaN,1951 FLATBUSH AVE,BROOKLYN,NY,11234,1128b5,20090319,0,0,NaN,2009,3
4,NaN,NaN,,"1ST COMMUNITY HEALTH CTR, LTD",CLINIC,NaN,NaN,0,NaN,3138 W CERMAK ROAD,CHICAGO,IL,60623,1128a1,19940524,0,0,NaN,1994,5


In [110]:
summary = LEIE.agg(['dtype', 'count', 'nunique', lambda x: x.isna().sum()]).rename(index={'<lambda>':'missing'}).T
summary.sort_values(by='missing', ascending=False)

,dtype,count,nunique,missing
wvrstate,object,13,10,72463
busname,object,3108,3052,69368
upin,object,6333,6156,66143
specialty,object,68276,192,4200
dob,float64,68499,20555,3977
lastname,object,69365,28194,3111
firstname,object,69366,11128,3110
address,object,72467,68551,9
state,object,72471,60,5
city,object,72475,9786,1


In [27]:
x = LEIE['npi'].value_counts()

In [29]:
x.value_counts()

1        5069
2          97
3           2
67207       1
Name: npi, dtype: int64

In [32]:
LEIE[LEIE['npi']!=0].shape

(5269, 20)

In [34]:
LEIE[LEIE['npi']==0].shape

(67207, 20)

In [35]:
LEIE.shape

(72476, 20)

In [36]:
LEIE.columns

Index(['lastname', 'firstname', 'midname', 'busname', 'general', 'specialty',
       'upin', 'npi', 'dob', 'address', 'city', 'state', 'zip', 'excltype',
       'excldate', 'reindate', 'waiverdate', 'wvrstate', 'exyear', 'exmonth'],
      dtype='object')

In [38]:
LEIE['excltype'] = LEIE['excltype'].str.strip()

## Matching npi

In [66]:
x = LEIE[LEIE['npi']!=0]
x[x.npi.duplicated()].shape #101 duplicated npis 

(101, 20)

In [53]:
LEIE[LEIE.npi!=0].shape[0] #5269 npis not zero

5269

In [46]:
LEIE.npi.nunique() #5169 unique npi

5169

In [37]:
#drop all duplicated npi
LEIE_uniquenpi = LEIE.drop_duplicates(subset='npi')
LEIE_uniquenpi.shape

(5169, 20)

In [39]:
npi_same = pd.merge(df, LEIE_uniquenpi, how='inner',on='npi').npi.values

In [40]:
df['Target'] = df.npi.isin(npi_same) #add label column

In [41]:
#only 278 npi's matched!
df[df.npi.isin(npi_same)].shape

(278, 29)

In [42]:
df.sample(5)

,npi,nppes_provider_last_org_name,nppes_provider_first_name,nppes_provider_mi,nppes_credentials,nppes_provider_gender,nppes_entity_code,nppes_provider_street1,nppes_provider_street2,nppes_provider_city,nppes_provider_zip,nppes_provider_state,nppes_provider_country,provider_type,medicare_participation_indicator,place_of_service,hcpcs_code,hcpcs_description,hcpcs_drug_indicator,line_srvc_cnt,bene_unique_cnt,bene_day_srvc_cnt,average_Medicare_allowed_amt,average_submitted_chrg_amt,average_Medicare_payment_amt,average_Medicare_standard_amt,Total_submitted_chrg_amt,Total_Medicare_payment_amt,Target
1022579,1104816115,NYBOER,JOHN,E,MD,M,I,10101 PARK ROWE AVE,SUITE 200,BATON ROUGE,708101686,LA,US,Physical Medicine and Rehabilitation,Y,F,99223,"Initial hospital inpatient care, typically 70 ...",N,74.0,65,73,197.310000,447.000000,140.595811,152.976622,33078.000000,10404.09,False
6821357,1699704916,MILLER,WILLIAM,D,MD,M,I,305 KEENE ST,STE. 203,COLUMBIA,652016897,MO,US,Family Practice,Y,O,99213,Established patient office or other outpatient...,N,33.0,33,33,68.938182,160.000000,27.670909,32.113333,5280.000000,913.14,False
6195965,1629318217,FLORES,ALICIA,L,PA-C,F,I,100 E ALTON GLOOR BLVD,NaN,BROWNSVILLE,785263328,TX,US,Physician Assistant,Y,F,99285,"Emergency department visit, problem with signi...",N,31.0,31,31,145.810000,2017.903226,95.632258,98.618710,62555.000000,2964.60,False
4218936,1427288943,MANGLA,JIMMI,NaN,NaN,M,I,200 1ST ST SW,NaN,ROCHESTER,559056712,MN,US,General Surgery,Y,F,99223,"Initial hospital inpatient care, typically 70 ...",N,54.0,54,54,210.500000,254.574074,165.030000,161.220000,13747.000000,8911.62,False
1027681,1104824366,ROBINSON,TRACEY,NaN,MD,F,I,2006 HEALTH CAMPUS DR,NaN,HARRISONBURG,228018679,VA,US,Rheumatology,Y,O,99214,Established patient office or other outpatient...,N,607.0,372,607,111.158731,281.169687,82.558600,82.378122,170670.000003,50113.07,False


In [108]:
LEIE.columns

Index(['LASTNAME', 'FIRSTNAME', 'MIDNAME', 'BUSNAME', 'GENERAL', 'SPECIALTY',
       'UPIN', 'NPI', 'DOB', 'ADDRESS', 'CITY', 'STATE', 'ZIP', 'EXCLTYPE',
       'EXCLDATE', 'REINDATE', 'WAIVERDATE', 'WVRSTATE', 'EXYEAR', 'EXMONTH'],
      dtype='object')

In [77]:
LEIE.busname.isnull().sum()

69368

## Imputing NPI in LEIE from CMS

In [ ]:
# filter where npi is 0
LEIE_0 = LEIE[LEIE['npi']==0]
LEIE_0.shape

In [73]:
df_indiv = df[df.nppes_entity_code=='I']
df_org = df[df.nppes_entity_code=='O']

In [116]:
LEIE_indiv = LEIE[LEIE.busname.isnull()]
LEIE_org = LEIE[LEIE.busname.notnull()]

#### Match entity code==O to Busnames